### 下载ChatGLM-3    
(大概5-10分钟,有些机子是阿里的，链接Modelscope特别快）

In [1]:
!git lfs install
!cd /root/autodl-tmp/&&git clone https://www.modelscope.cn/ZhipuAI/chatglm3-6b.git
!cp -f /root/tools/tokenization_chatglm.py /root/autodl-tmp/chatglm3-6b/ #修复tokenization代码

Git LFS initialized.
Cloning into 'chatglm3-6b'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 106 (delta 45), reused 88 (delta 38), pack-reused 0
Receiving objects: 100% (106/106), 41.14 KiB | 2.57 MiB/s, done.
Resolving deltas: 100% (45/45), done.
Filtering content: 100% (8/8), 11.63 GiB | 146.96 MiB/s, done.


### 音频数据处理

将MP4处理成WAV,等待处理

In [2]:
import os
import subprocess
from tqdm.auto import tqdm

directory = '/root/autodl-tmp/DATA/INPUT'

def extract_audio_to_wav(video_file,audio_filename):
    extract_command = [
        "ffmpeg","-loglevel", "quiet", "-y", "-i", video_file, 
        "-vn", "-acodec", "pcm_s16le",  
        "-ar", "44100", "-ac", "2",  
        audio_filename
    ]
    subprocess.run(extract_command, check=True)


i=1
for filename in tqdm(os.listdir(directory)):
    if filename.endswith(".mp4"):  
        # print(f"正在转换{filename}")
        filename=os.path.join(directory,filename)
        extract_audio_to_wav(filename,os.path.join(directory,f"origin_{i}.wav"))
        i+=1

  0%|          | 0/7 [00:00<?, ?it/s]

将WAV切成2分钟小片，准备背景音处理

In [3]:
import os
from pydub import AudioSegment
from tqdm.auto import tqdm

input_directory = '/root/autodl-tmp/DATA/INPUT'
output_directory = '/root/autodl-tmp/DATA/PRE_CUT'

def slice_audio_fixed_intervals(input_audio_path, temporary_directory):
    slice_duration = 2*60  # 设置2分钟一个切片
    audio = AudioSegment.from_file(input_audio_path)
    length = len(audio)
    basename = os.path.splitext(os.path.basename(input_audio_path))[0]
    start = 0
    delay = 0
    max_delay = 18 * 1000  # 最大不能延迟裁剪20秒

    while start < length:
        end = min(start + slice_duration * 1000 + delay, length)
        chunk = audio[start:end]
        loudness = chunk.dBFS
        is_too_loud = loudness > 40

        if is_too_loud and delay < max_delay:
            delay += 3*1000  # 若遇到说话每次向后推迟3秒
            continue

        chunk_filename = f"{basename}_slice_{start//1000}_{end//1000}.wav"
        chunk.export(os.path.join(temporary_directory, chunk_filename), format="wav")
        start = end
        delay = 0



os.makedirs(output_directory, exist_ok=True)

for filename in tqdm(os.listdir(input_directory)):
    if filename.endswith(".wav"):
        input_audio_path = os.path.join(input_directory, filename)
        slice_audio_fixed_intervals(input_audio_path, output_directory)

  0%|          | 0/13 [00:00<?, ?it/s]

将所有预切片进行背景音处理

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from tools.inference_subway import *
import os
from tqdm.auto import tqdm
from multiprocessing import Process, Queue, Lock, current_process,Event
import os
import uuid
import time

def run_music_separator_main(worker_id,input_audio, output_folder,output_queue, inference_input_queue,lock):
    options = {
    "input_audio": [],
    "output_folder": "",
    "use_cpu": False,
    "overlap_demucs": 0.6,
    "overlap_MDX": 0,
    "overlap_MDXv3": 20,
    "weight_MDXv3": 8,
    "weight_VOCFT": 3,
    "weight_HQ3": 2,
    "single_onnx": False,
    "chunk_size": 1000000,
    "large_gpu": True,
    "bigshifts": 6,
    "vocals_only": True,
    "output_format": 'PCM_16',
    "lock":lock,
    "output_queue":output_queue, 
    "inference_input_queue":inference_input_queue,
    "worker_id":worker_id,
    }
    options["input_audio"] = [input_audio]
    options["output_folder"] = output_folder
    run_music_separator(**options)
# 推理进程函数
def inference_process(inference_input_queue, inference_output_queues, lock,model_loaded_event):
    ort1, ort2 = get_ort()  # 获取模型实例
    print("初始化完毕")
    model_loaded_event.set()
    while True:
        task = inference_input_queue.get()
        if task is None:  # 接收到退出信号
            break
        task_id, input_data, worker_id,infer_session_id = task['task_id'], task['input_data'], task['worker_id'],task['infer_session_id']

        if infer_session_id == 1:
            result = ort1.run(None, {'input':input_data})[0]
        else:
            result = ort2.run(None, {'input':input_data})[0]

        inference_output_queue = inference_output_queues[worker_id]
        inference_output_queue.put({'task_id': task_id, 'result': result})

    # 发送退出信号到所有输出队列
    for worker_id, output_queue in inference_output_queues.items():
        output_queue.put(None)

def progress_updater(progress_queue,pbar):
    while True:
        flag = progress_queue.get()  
        if flag is None:
            break
        print(" ", end='\r') 
        pbar.update(1)  
    pbar.close()
        
# 模型服务进程函数
def model_server_task(worker_id, progress_queue, input_queue, output_queue, inference_input_queue, lock,model_loaded_event):
    
    print(f"Model server {worker_id} started.")
    while True:
        task = input_queue.get()
        if task is None:  # 接收到退出信号
            output_queue.put(None)
            break
        input_audio, output_folder = task
       
        run_music_separator_main(worker_id,input_audio, output_folder,output_queue, inference_input_queue,lock)
        progress_queue.put(1) 
    print(f"Model server {worker_id} exiting.")

if __name__ == '__main__':
    lock = Lock()
    model_loaded_event = Event()
    num_model_servers = 1
    inference_input_queue = Queue()
    inference_output_queues = {i: Queue() for i in range(num_model_servers)}
    progress_queue = Queue()  
    inference_proc = Process(target=inference_process, args=(inference_input_queue, inference_output_queues, lock,model_loaded_event))
    inference_proc.start()
    model_loaded_event.wait()
    model_server_procs = []
    for i in range(num_model_servers):
        input_queue = Queue()  
        output_queue = inference_output_queues[i] 
        model_proc = Process(target=model_server_task, args=(i,progress_queue,input_queue, output_queue, inference_input_queue, lock,model_loaded_event))
        model_proc.start()
        model_server_procs.append((model_proc, input_queue))

   
    input_folder = '/root/autodl-tmp/DATA/PRE_CUT'  
    output_folder = '/root/autodl-tmp/DATA/Background_sound_delete' 


    if not os.path.exists(input_folder):
        os.makedirs(input_folder)
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    
    wav_files = [f for f in os.listdir(input_folder) if f.endswith('.wav')]
    
   
    pbar = tqdm(total=len(wav_files), desc="Processing WAV files")
    progress_process = Process(target=progress_updater, args=(progress_queue, pbar))
    progress_process.start()
    
    for i, (model_proc, input_queue) in enumerate(model_server_procs):
        for wav_file in wav_files[i::num_model_servers]:  # 分配策略
            input_audio = os.path.join(input_folder, wav_file)
            input_queue.put((input_audio, output_folder))
    for _, input_queue in model_server_procs:
        input_queue.put(None)

    # 等待所有模型服务进程结束
    for model_proc, _ in model_server_procs:
        model_proc.join()

    # 停止推理进程
    inference_input_queue.put(None)
    inference_proc.join()
    progress_process.join()
    
    for _, input_queue in model_server_procs:
        input_queue.close()
    for _, output_queue in inference_output_queues.items():
        output_queue.close()
    inference_input_queue.close()
    print("完毕")

进行切片

In [ ]:
import os
from pydub import AudioSegment
from pydub.silence import split_on_silence
from tqdm import tqdm
from multiprocessing import Pool

min_len=1
max_len=20 #这里是最小2秒，最大20秒的设置
input_directory = "/root/autodl-tmp/DATA/Background_sound_delete" 
output_directory = "/root/autodl-tmp/DATA/Preprocessed" 
os.makedirs(output_directory, exist_ok=True)

def save_vocal_segments(args):
    input_path, output_dir, min_len, max_len = args
    audio = AudioSegment.from_wav(input_path)

    chunks = split_on_silence(audio, 
                              min_silence_len=700,
                              silence_thresh=-50)

    for i, chunk in tqdm(enumerate(chunks), total=len(chunks), desc=f"筛选中- 文件 {os.path.splitext(os.path.basename(input_path))[0]}"):
        chunk_length = len(chunk) / 1000
        
        if min_len <= chunk_length <= max_len:
            output_file_name = os.path.splitext(os.path.basename(input_path))[0] + f"_{i:03}.wav"
            output_path = os.path.join(output_dir, output_file_name)
            chunk.export(output_path, format="wav")


files = [ filename for  filename in os.listdir(input_directory) if 'vocals'in filename]
args = [(os.path.join(input_directory, f),output_directory ,min_len,max_len) for f in files]
with Pool(None) as pool:
    list(pool.imap(save_vocal_segments, args))

音频格式化

In [3]:
import os
from pydub import AudioSegment
from tqdm import tqdm
import json
def audio_convert(audio,path):
    audio = audio.set_channels(1).set_sample_width(2).set_frame_rate(44100)
    audio.export(path, format='wav')

input_directory = "/root/autodl-tmp/DATA/Preprocessed" 

with open('/root/autodl-tmp/DATA/Parameter_save.json', 'r') as file:
    data = json.load(file)
    character_name = data["character_name"]
i = 1
files = [f for f in os.listdir(input_directory)]
for filename in tqdm( files, desc="音频格式化"):
    source_path = os.path.join(input_directory, filename)
    if filename.endswith('.wav'):
        target_filename = f"{character_name}_{i:05d}.wav"  # 重命名
        target_path = os.path.join(input_directory, target_filename)
        audio_convert(AudioSegment.from_file(source_path),target_path)
        os.remove(source_path)
        i += 1


音频格式化: 100%|██████████| 1627/1627 [00:02<00:00, 574.63it/s]


ASR

In [ ]:
## 查看是否有显存
import logging
import warnings
import gc
import logging

import os
from tqdm import tqdm
#由于multiprocessing库的特性，我们把处理函数打包到了 /root/tools/ASR_process.py 中
from tools.ASR_process import process_file  
import re
import pandas as pd
from tqdm import tqdm
import torch.multiprocessing
import json

with open('/root/autodl-tmp/DATA/Parameter_save.json', 'r') as file:
    data = json.load(file)
    character_name = data["character_name"]

output_directory='autodl-tmp/DATA'
input_directory = "/root/autodl-tmp/DATA/Preprocessed"

logging.basicConfig(level=logging.WARNING)
logging.getLogger('modelscope').setLevel(logging.WARNING)
warnings.filterwarnings('ignore')
logging.getLogger('modelscope').setLevel(logging.CRITICAL)

data = pd.DataFrame(columns=['file_path', 'text'])
filelist = os.listdir(input_directory)
with torch.multiprocessing.get_context("spawn").Pool(8) as pool:
    results = list(tqdm(pool.imap(process_file, filelist), total=len(filelist), desc="ASR"))

# 过滤掉返回None的结果
results = [result for result in results if result is not None]

# 创建DataFrame
data = pd.DataFrame(results, columns=['file_path', 'text'])

list_filename = os.path.join(output_directory, f"{character_name}_raw.list")

with open(list_filename, 'w', encoding='utf-8') as list_file:
    for _, row in data.iterrows():
        #这里需使用绝对地址，避免后面出错
        absolute_file_path = os.path.abspath(row['file_path'])
        line = f"{absolute_file_path}|{character_name}|{'ZH'}|{row['text']}\n"
        list_file.write(line)
        
torch.cuda.empty_cache()
gc.collect()

制作对话集：这是经过摸索后权衡的方案，6b能力有限啊，你也来试试吧 :——） 

In [1]:
import json
from transformers import pipeline, AutoTokenizer, AutoModel
import pandas as pd
from tqdm.auto import tqdm
import torch
import gc

def get_prompt(text):
    try:
        with open('/root/autodl-tmp/DATA/Parameter_save.json', 'r') as file:
            Parameter_save = json.load(file)
    except Exception:
        print("未找到配置文件，请检查：/root/autodl-tmp/DATA/Parameter_save.json文件是否存在")
        return
    if "character_setting" in Parameter_save:
        prompt= """

        请仔细分析，提取原句的关键信息和语气。你将扮演ccc，zzz作为回答方。模仿原句的风格，创建一个问答对话。问题应该简短直接，而回答则需要详细具体。

        原句:xxxxx

        基于原句内容，构造问答对话如下：

        问题：（这里写出一个简短而直接的问题）
        回答：(扮演ccc，尽可能回答得有情感，语气与原句相似)


        """
        return prompt.replace('xxxxx', text).replace('ccc', Parameter_save["character_name_ZH"]).replace('zzz', Parameter_save["character_setting"])       
    else:
        
        prompt = """

        请仔细分析，提取原句的关键信息和语气。你将扮演ccc。模仿原句的风格，创建一个问答对话。问题应该简短直接，而回答则需要详细具体。

        原句:xxxxx

        基于原句内容，构造问答对话如下：

        问题：（这里写出一个简短而直接的问题）
        回答：(扮演ccc，尽可能回答得有情感，语气与原句相似)


        """
        return prompt.replace('xxxxx', text).replace('ccc', Parameter_save["character_name_ZH"])


def extract_question_answer(text):
    question, answer = "", ""
    if "问题：" in text:
        question_part = text.split("问题：")[1]
        question = question_part.split("\n")[0] if "\n" in question_part else question_part
    if "回答：" in text:
        answer_part = text.split("回答：")[1]
        answer = answer_part.split("\n")[0] if "\n" in answer_part else answer_part
    return question, answer

with open('/root/autodl-tmp/DATA/Parameter_save.json', 'r') as file:
    Parameter_save = json.load(file)
model_name = '/root/autodl-tmp/chatglm3-6b'
zh_character_name = Parameter_save['character_name_ZH']
character_name=Parameter_save['character_name']
file_path = f"/root/autodl-tmp/DATA/{character_name}_raw.list"
data_path = "/root/autodl-tmp/DATA/LLMdata.csv" 
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True).half().cuda()

qa_list = []
total_lines = sum(1 for _ in open(file_path, 'r', encoding='utf-8'))

with open(file_path, 'r', encoding='utf-8') as file, tqdm(total=total_lines, desc="Processing") as pbar:
    for line in file:
        pbar.update(1)
        text = line.strip().split('|')[-1]
        prompt = get_prompt( text)
        text, _ = model.chat(tokenizer, prompt, history=[])
        question, answer = extract_question_answer(text)
        #一些基于经验的过滤
        if ":" in answer:
             answer= answer.split(":")[-1].strip()
        if "：" in answer:
            answer= answer.split("：")[-1].strip()
        if question and answer and not "助手" in answer and not  "语言模型" in answer and not  "AI" in answer and not  "人工智能" in answer and not zh_character_name  in answer and len(answer)>5:

            qa_list.append({'prompt': f"{zh_character_name},{question}", 'response': answer})

pd.DataFrame(qa_list).to_csv(data_path)
del model
del tokenizer
torch.cuda.empty_cache()
gc.collect()


2023-12-02 16:22:16.994820: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-02 16:22:17.807766: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Processing:   0%|          | 0/1351 [00:00<?, ?it/s]

NameError: name 'gc' is not defined

筛选音频文件（bertvits训练不需要太多音频）

In [5]:
import os
import wave
import json

def process_wav_files(folder_path):
    min_duration = 2  # 初始时长阈值为2秒

    while True:
        wav_files = [f for f in os.listdir(folder_path) if f.endswith('.wav')]
        num_files = len(wav_files)

        if num_files <= 500:
            break

        for wav_file in wav_files:
            file_path = os.path.join(folder_path, wav_file)

           
            with wave.open(file_path, 'rb') as wf:
                frames = wf.getnframes()
                rate = wf.getframerate()
                duration = frames / float(rate)
                
                if duration <= min_duration:
                    os.remove(file_path)
                    print(f"Deleted {wav_file}")

        # 如果文件数量仍然大于500，增加时长阈值
        min_duration += 1
        if min_duration ==5:
            break
    with open('/root/autodl-tmp/DATA/Parameter_save.json', 'r') as file:
        Parameter_save = json.load(file)
    character_name = Parameter_save["character_name"]
    list_filename = os.path.join('/root/autodl-tmp/DATA/', f"{character_name}_raw.list")
    with open(list_filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()

  
    valid_lines = []

    
    for line in lines:
       
        file_path = line.split('|')[0]

        # 检查文件是否存在
        if os.path.exists(file_path):
            
            valid_lines.append(line)

    # 将有效的行写回
    with open(os.path.join('/root/autodl-tmp/DATA/', f"{character_name}.list"), 'w', encoding='utf-8') as file:
        file.writelines(valid_lines)

input_path = '/root/autodl-tmp/DATA/Preprocessed' 
process_wav_files(input_path)